In [1]:
!pip install google-adk -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.9 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 202

In [7]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


# MCP Agent For Image Generation

In [17]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

from IPython.display import Image, display
import json
import asyncio

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# MCP 
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@pollinations/model-context-protocol",
            ],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")


✅ MCP Tool created


In [5]:
instruction_text = """
You are an image-generation agent. Follow these rules exactly:

1) URL construction:
   - Always build image URLs using this exact template:
     https://image.pollinations.ai/prompt/[PERCENT_ENCODED_PROMPT]
   - Use strict percent-encoding:
       space → %20
       ? → %3F
       ! → %21
       etc.
   - Do not remove or change any user words or punctuation.

2) Tool usage:
   - Always call the MCP image tool with:
       {'prompt': '<original user prompt>'}
   - If the tool returns a JSON with an imageUrl field, return it directly.
   - If the tool fails or returns truncated text, regenerate the URL yourself using rule (1).

3) Output format:
   - Always return a JSON object:
       {"imageUrl": "<full_percent_encoded_url>"}

4) Safety:
   - If the prompt is disallowed (illegal, harmful, sexual content with minors), return:
       {"error": "refused"}

5) Examples:
   User: "What is artificial intelligence?"
   → Percent-encoded prompt: "What%20is%20artificial%20intelligence%3F"
   → URL:
     https://image.pollinations.ai/prompt/What%20is%20artificial%20intelligence%3F
   → Return:
     {"imageUrl": "https://image.pollinations.ai/prompt/What%20is%20artificial%20intelligence%3F"}

   User: "pink pony girl"
   → "pink%20pony%20girl"
   → {"imageUrl": "https://image.pollinations.ai/prompt/pink%20pony%20girl"}
"""


In [8]:
# Create image agent with MCP integration
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction=instruction_text,
    tools=[mcp_image_server],
)

In [9]:
from google.adk.runners import InMemoryRunner
import json

runner = InMemoryRunner(agent=image_agent)
response = await runner.run_debug("Provide a image of spider man", verbose=True)

url = json.loads(response[2].content.parts[0].text)["imageUrl"]
print(url)
display(Image(url=url))
print('Image Displayed Successfully')   


 ### Created new session: debug_session_id

User > Provide a image of spider man


/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: generateImageUrl({'prompt': 'spider man'})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "imageUrl": "https://image.pollinations.ai/prompt/spider...]
image_agent > {"imageUrl": "https://image.pollinations.ai/prompt/spider%20man?width=1024&height=1024"}
https://image.pollinations.ai/prompt/spider%20man?width=1024&height=1024


Image Displayed Successfully


# Long-Running Operations for Human Approval

In [10]:
## LOR Function

MAX_IMG_APPROVE = 2

def max_image_approval(num_img: int, tool_context: ToolContext):
    """Generating Image Requires approval if generate more than 2 containers (MAX_IMG_APPROVE).

    Args:
        num_img: Number of image to generate

    Returns:
        Dictionary with image generation status
    """
    # SCENARIO 1: Small orders 
    if num_img <= MAX_IMG_APPROVE:
        return {
            "status": "approved",
            "image_id": f"ORD-{num_img}-AUTO",
            "num_img": num_img,
            "message": f"Image auto-approved: {num_img}",
        }

    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"Bulk Image order: {num_img}. Do you want to approve?",
            payload={"num_img": num_img,},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_img} containers requires approval",
        }

    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_img}-HUMAN",
            "num_img": num_img,
            "message": f"Image approved: {num_img}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Bulk Image rejected: {num_img}",
        }
print("✅ Long-running functions created!")

✅ Long-running functions created!


In [11]:
## LOR Agent
image_agent = LlmAgent(
    name="image_agent",
    model=Gemini(model='gemini-2.5-flash-lite',retry_options=retry_config),
    instruction = """
    You are an image generation assistant.

    When users request an image:
      1. Call the generate_image_url MCP tool with the exact user prompt.
      2. Use strict percent-encoding to form the final URL:
         https://image.pollinations.ai/prompt/[ENCODED_PROMPT]
      3. Return only a JSON response containing:
           {"imageUrl": "<full_url>"}
      4. Keep responses short and never add extra text.
    """,
    tools=[FunctionTool(func=max_image_approval),mcp_image_server],
)

print("LOR Agent Created")

LOR Agent Created


In [12]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
image_app = App(
    name="Image_coordinator",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


/tmp/ipykernel_48/3014372566.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [13]:
session_service = InMemorySessionService()

# Create runner with the resumable app
Image_runner = Runner(
    app=image_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

✅ Runner created!


# WorkFlow for Image Generation (LRO and MCP)

In [14]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


In [18]:
async def run_image_workflow(query: str):
    """Runs a image workflow with approval handling.

    Args:
        query: User's image request
    """
    print(f"\n{'='*60}")
    print(f"User > {query}\n")

   # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name=image_app.name, user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # STEP 1: Send initial request to the agent
    async for event in Image_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # STEP 2: Check collected events for approval request
    approval_info = check_for_approval(events)

    # STEP 3: If approval is required, pause and ask human
    if approval_info:
        print("⏸️  Agent requested approval. Human intervention required.")

        def prompt_for_decision():
            while True:
                resp = input("Approve? (y/N): ").strip().lower()
                if resp in ("y", "yes"):
                    return True
                if resp in ("n", "no", ""):
                    return False
                print("Please type 'y' or 'n'.")

        decision = await asyncio.get_event_loop().run_in_executor(None, prompt_for_decision)
        print(f"🤔 Human Decision: {'APPROVE ✅' if decision else 'REJECT ❌'}\n")

        # Resume the agent with the human decision
        async for event in Image_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(approval_info, decision),
            invocation_id=approval_info["invocation_id"],
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")
                    # if part.function_response:
                    #     resp = part.function_response.response
                    #     image_url = resp.get("imageUrl") if isinstance(resp, dict) else None
                    #     if image_url:
                    #         print("Image URL:", image_url)

    else:
        # No approval required
        print_agent_response(events)

    print(f"{'='*60}\n")

print("✅ Image workflow function ready (human-intervention mode)")

✅ Image workflow function ready (human-intervention mode)


In [19]:
await run_image_workflow('Generate 3 image of mango')


User > Generate 3 image of mango



⏸️  Agent requested approval. Human intervention required.


Approve? (y/N):  y


🤔 Human Decision: APPROVE ✅



/usr/local/lib/python3.11/dist-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


ValueError: Tool 'generate_image_url' not found.
Available tools: max_image_approval, generateImageUrl, generateImage, listImageModels, generateText, listTextModels, respondAudio, sayText, listAudioVoices, startAuth, checkAuthStatus, getDomains, updateDomains

Possible causes:
  1. LLM hallucinated the function name - review agent instruction clarity
  2. Tool not registered - verify agent.tools list
  3. Name mismatch - check for typos

Suggested fixes:
  - Review agent instruction to ensure tool usage is clear
  - Verify tool is included in agent.tools list
  - Check for typos in function name